#  SETUP

In [1]:
# if the script finishes very quickly (and generates an empty excel file), click run again
# if the script errors on the "Login Cell" (added a comment to indicate which cell that is below), set IS_HEADLESS to "False" and run again. The scraper will automatically launch a page and attempt to login to LinkedIn. It's likely erroring because LinkedIn is asking for a captcha to verify the user is not a bot. Solve the captch/challenge and login. Once successfully logged in, set IS_HEADLESS back to "True" and run again.

IS_HEADLESS = False
# CHANGE THIS TO YOUR LINKEDIN ACCOUNT INFO
LINKEDIN_EMAIL = "XXXXXXXX"
LINKEDIN_PASSWORD = "XXXXXXXXX"
COMPANY_CATEGORIES = {
    "SECURITY": [
        "Okta",
        "Snyk",
        "R2C/Semgrep", 
        "Wiz",
        "Lacework",
        "Crowdstrike",
        "Palo Alto Networks",
        "Island",
        "Vanta",
        "Material Security",
        "Abnormal Security",
        "Samsara",
    ],
    "OTHER": [
        "Figma",
        "Airtable",
        "Notion",
        "Canva",
        "Webflow",
        "Faire",
        "Deel",
        "Rippling",
        "Flexport",
        "Benchling",
        "Solugen"
    ],
    "PUBLIC": [
        "Doordash",
        "Uber",
        "Palantir",
        "Airbnb",
        "Instacart"
    ],
    "INFRA": [
        "Fivetran",
        "DBT",
        "Temporal",
        "Cockroach Labs",
        "Grafana",
        "Zapier",
        "Starburst",
        "Retool",
        "Sentry",
        "Sourcegraph",
        "Cribl",
        "Vercel",
        "Clickhouse",
        "Github,"
        "Cisco Meraki",
    ],
    "FINTECH": [
        "Robinhood",
        "Square",
        "Stripe",
        "Ramp",
        "Brex",
        "Plaid",
        "Modern Treasury",
        "Mercury",
        "Persona",
        "Klarna",
        "Nubank"
    ],
    "CRYPTO": [
        "Coinbase",
        "Uniswap",
        "Chainalysis",
        "Arbitrum",
        "TRM",
        "Fireblocks",
        "Eigenlayer"
    ],
    "FRONTIER": [
        "Anduril",
        "SpaceX",
        "Zipline",
        "Varda",
        "Hadrian"
    ],
    "AI": [
        "Bytedance",
        "Scale AI",
        "Anthropic",
        "Robust intelligence",
        "OpenAI",
        "Predibase",
        "Cohere",
        "Databricks",
        "Hugging Face",
        "RunwayML",
        "Tecton",
        "Weights & Biases",
        "Kumo AI",
        "NVIDIA",
        "Adept",
        "Glean",
        "Character.ai",
        "Midjourney",
        "Facebook AI",
        "FAIR",
        "Google brain"
    ]
}

In [2]:
import pickle
import os
import random
import time
from linkedin_scraper import actions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

# LOGIN

In [3]:
# IF THIS CELL ERRORS DUE TO CAPTCHA, do this:
# manually complete the captcha, click on the next cell, and select Run menu, select "run selected cell and all below" 

def instantiate_driver():
    options = FirefoxOptions()
    if IS_HEADLESS:
        options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    
    try:
        actions.login(driver, LINKEDIN_EMAIL, LINKEDIN_PASSWORD)
    except Exception as e:
        print("Error logging in. Please complete the captcha challenge and login manually.")
        print(e)

    time.sleep(15)
    return driver
    
driver = instantiate_driver()

Error logging in. Please complete the captcha challenge and login manually.
Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



# SALES NAV HELPER FUNCTIONS

In [4]:
class ScrapedProfile:
    def __init__(self, profile_name, experiences, profile_school, profile_dist, mutuals, profile_description, profile_link):
        self.profile_name = profile_name
        self.experiences = experiences
        self.profile_school = profile_school
        self.profile_dist = profile_dist
        self.mutuals = mutuals
        self.profile_description = profile_description
        self.profile_link = profile_link

def check_pickles(historyPickle, toScrapePickle):
    scraped_urls = []
    with open(historyPickle, 'rb') as f:
        try:
            scraped_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading history pickle file")
        print(len(scraped_urls))

    to_scrape_urls = []
    with open(toScrapePickle, 'rb') as f:
        try:
            to_scrape_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading to scrape url pickle file")
        print(len(to_scrape_urls))

    return scraped_urls, to_scrape_urls

def deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls):
    to_scrape_urls = list(set(to_scrape_urls))
    originalCount = len(to_scrape_urls)
    to_scrape_urls = [url for url in to_scrape_urls if url not in already_scraped_urls]
    print(f"Removed {originalCount - len(to_scrape_urls)} duplicates")

    with open(toScrapePickle, 'wb') as to_scrape_file:
        pickle.dump(to_scrape_urls, to_scrape_file)
    
    print(len(to_scrape_urls))


"""
Wait for an element to be present on the page and return it.

Parameters:
- driver: The WebDriver instance
- by: The method to locate the element (default: By.CLASS_NAME)
- name: The name or identifier of the element to wait for
- base: The base element to search from (default: None, which uses the driver)
- timeout: Maximum time to wait for the element (default: 180 seconds)

Returns:
- The WebElement if found
- None if the element is not found within the timeout period
"""
def wait_for_element_to_load(driver, by=By.CLASS_NAME, name="pv-top-card", base=None, timeout=10):
    base = base or driver
    try:
        element = WebDriverWait(base, timeout).until(
            EC.presence_of_element_located((by, name))
        )
        return element
    except TimeoutException:
        print(f"Timed out waiting for element: {by}={name}")
        return None
    except Exception as e:
        print(f"An error occurred while waiting for element {by}={name}: {str(e)}")
        return None

# Scrolls the page to bring the specified profile element into view
# Returns True if successful, False if an error occurs
def scroll_to_profile(driver, profile):
    try:
        driver.execute_script("arguments[0].scrollIntoView();", profile)
        return True
    except Exception as e:
        print(f"Error scrolling to profile: {e}")
        return False

# Clicks on the profile element to open its details
# Returns True if successful, False if an error occurs
def click_profile(profile):
    try:
        salesNavOpenProfileButton = profile.find_element(By.CLASS_NAME, "artdeco-entity-lockup__title")
        salesNavOpenProfileButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking profile: {e}")
        return False

# Clicks the three dots button to open the dropdown menu
# Returns True if successful, False if an error occurs
def click_three_dots_button(driver):
    try:
        actionContainer = driver.find_element(By.CLASS_NAME, "_actions-container_1dg5u8")
        threeDotsButton = actionContainer.find_element(By.CLASS_NAME, "_icon_ps32ck")
        threeDotsButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Error clicking three dots button")
        return False

# Retrieves the LinkedIn URL from the dropdown menu
# Returns the URL if successful, None if an error occurs
def get_linkedin_url(driver):
    try:
        dropdownContainer = driver.find_element(By.CLASS_NAME, "_visible_x5gf48")
        normalLinkedInUrl = dropdownContainer.find_elements(By.TAG_NAME, "a")[1].get_attribute("href")
        return normalLinkedInUrl
    except (NoSuchElementException, IndexError) as e:
        print(f"Error getting LinkedIn URL: {e}")
        return None

# Closes the profile popout
# Returns True if successful, False if an error occurs
def close_popout(driver):
    try:
        header = driver.find_element(By.CLASS_NAME, "_inline-sidesheet-header-actions_1cn7lg")
        button = header.find_elements(By.CLASS_NAME, "_button_ps32ck")[1]
        button.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException, IndexError) as e:
        print(f"Error closing popout: {e}")
        return False

# Navigates to the next page of search results
# Returns True if successful, False if there are no more pages or an error occurs
def navigate_to_next_page(driver):
    try:
        nextPageButton = driver.find_element(By.CLASS_NAME, "artdeco-pagination__button--next")
        nextPageButton.click()
        return True
    except (NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"No more pages or error navigating: {e}")
        return False

# Main scraping function
def scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle):
    # reinstantiate_driver(driver)
    driver.get(SALES_NAV_SEARCH_URL)
    
    while True:
        wait_for_element_to_load(driver, By.ID, "search-results-container")
        profiles = driver.find_elements(By.CLASS_NAME, "artdeco-list__item")
        
        for profile in profiles:
            if not scroll_to_profile(driver, profile):
                continue

            if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-entity-lockup__title"):
                continue

            if not click_profile(profile):
                continue

            if not wait_for_element_to_load(driver, By.CLASS_NAME, "_actions-container_1dg5u8"):
                continue

            # time.sleep(2 + random.random() * 6)

            if not click_three_dots_button(driver):
                continue

            if not wait_for_element_to_load(driver, By.CLASS_NAME, "_visible_x5gf48"):
                continue

            # time.sleep(2 + random.random() * 6)

            normalLinkedInUrl = get_linkedin_url(driver)
            if normalLinkedInUrl:
                if normalLinkedInUrl in already_scraped_urls:
                    print("Skipping (already scraped): " + normalLinkedInUrl)
                else:
                    to_scrape_urls.append(normalLinkedInUrl)
                    with open(toScrapePickle, 'wb') as f:
                        pickle.dump(to_scrape_urls, f)
                    print("Successfully scraped: " + normalLinkedInUrl)

            if not close_popout(driver):
                continue

        next_button = wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-pagination__button--next")
        if not next_button or not next_button.is_enabled():
            break

        next_button.click()

        if not wait_for_element_to_load(driver, By.CLASS_NAME, "artdeco-list__item"):
            break

    return to_scrape_urls

# PROFILE SCRAPING HELPER FUNCTIONS

In [5]:
def filter():
    pass     
    # try:
        #     WebDriverWait(driver, 240).until(lambda d: d.execute_script('return document.readyState') == 'complete')
        # except:
        #     driver = reinstantiate_driver()

        # # FILTERING
        
        # likely_founder = True
        # relevant_exp = True

        # cur_exp = experiences[0]
        # relevant_companies = ["stealth", "new"]
        # if any(company in cur_exp.institution_name.split(" ·")[0].lower() for company in relevant_companies) or "present" not in cur_exp.to_date.lower():
        #     likely_founder = True

        # relevant_titles = ["product", "engineer", "sales", "business development", "founder", "head", "lead", "senior", "staff", "chief", "growth"]
        # for experience in experiences[1:5]:
        #     if any(title in experience.position_title.lower() for title in relevant_titles):
        #         relevant_exp = True
        #         break
        # relevant_exp = True

        # if not (likely_founder and relevant_exp):
        #     print(likely_founder, relevant_exp)
        #     return None

        # person_obj = Person(profile_link, driver = driver, scrape=False, experiences = [None])
        # try:
        #     WebDriverWait(driver, 240).until(lambda d: d.execute_script('return document.readyState') == 'complete')
        # except:
        #     driver = reinstantiate_driver(driver)
        # time.sleep(2 + random.random() * 7)
        
# Helper function to scrape experiences
# Returns scraped experiences if successful, otherwise returns empty list
def get_experiences(driver):
    scraped_experiences = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        experience_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")

        if len(experience_items) > 0:
            print(f"Found {len(experience_items)} experience items")
            for item in experience_items:
                hidden_spans = item.find_elements(By.CSS_SELECTOR, "span.visually-hidden")
                experience_texts = [span.text for span in hidden_spans]
                experience = {
                    "title: ": experience_texts[0],
                    "company: ": experience_texts[1],
                    "dates: ": experience_texts[2],
                }
                
                if len(experience_texts) > 3:
                    experience["location: "] = experience_texts[3]
                if len(experience_texts) > 4:
                    experience["summary: "] = experience_texts[4]
                if len(experience_texts) > 5:
                    experience["remaining: "] = (", ").join(experience_texts[5:])

                scraped_experiences.append(experience)

            for experience in scraped_experiences:
                for k, v in experience.items():
                    print(f"{k}: {v}")
            print("Successfully scraped experiences")
        else:
            print("No experiences found")

    except Exception as e:
        print(e)
        print("ERROR: No experiences found")

    return scraped_experiences


# Helper function to scrape education
# Returns scraped education if successful, otherwise returns empty list
def get_education(driver):
    scraped_education = []

    try:
        wait_for_element_to_load(driver, By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")
        education_items = driver.find_elements(By.CSS_SELECTOR, "li.pvs-list__paged-list-item.artdeco-list__item.pvs-list__item--line-separated.pvs-list__item--one-column")

        if len(education_items) > 0:
            for item in education_items:
                hidden_spans = item.find_elements(By.CSS_SELECTOR, "span.visually-hidden")
                education_texts = [span.text for span in hidden_spans]
                education = {
                    "school: ": education_texts[0],
                }
                
                if len(education_texts) > 1:
                    education["degree: "] = education_texts[1]
                if len(education_texts) > 2:
                    education["dates: "] = education_texts[2]
                if len(education_texts) > 3:
                    education["remaining: "] = (", ").join(education_texts[5:])

                scraped_education.append(education)

            for education in scraped_education:
                for k, v in education.items():
                    print(f"{k}: {v}")
            print("Successfully scraped education")
        else:
            print("No education found")

    except Exception as e:
        print(e)
        print("ERROR: No education found")

    return scraped_education


# Helper function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def scrape_degree_of_connection_and_mutuals(driver):
    scraped_profile_dist = "4+"
    scraped_mutuals = "N/A"

    try:
        scraped_profile_dist = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.dist-value").text
        if scraped_profile_dist == "1st" or scraped_profile_dist == "2nd":
            wait_for_element_to_load(driver, By.CSS_SELECTOR, "h1.text-heading-xlarge.inline.t-24.v-align-middle.break-words")
            try:
                span_element = wait_for_element_to_load(driver, By.CSS_SELECTOR, "span.t-normal.t-black--light.t-14.hoverable-link-text")
                scraped_mutuals = span_element.text.split('\n')[0]
                print("Successfully found mutual connections: " + scraped_mutuals)
            except:
                print("ERROR: mutuals not found")
        print("Successfully scraped degree of connection: " + scraped_profile_dist)

    except Exception as e:
        print(e)
        print("No degree of connection found")

    return scraped_profile_dist, scraped_mutuals

# Helper function to scrape description
# Returns scraped description if successful, otherwise returns N/A
def scrape_description(driver):
    scraped_description = "N/A"

    try:
        scraped_description = driver.find_element(By.CLASS_NAME, "text-body-medium.break-words").text
        print("Successfully scraped description: " + scraped_description)
    except Exception as e:
        print(e)
        print("ERROR: description not found")


# Main function to scrape degree of connection and mutuals
# Returns scraped degree of connection and mutuals if successful, otherwise returns N/A
def scrape_profile(driver, scraped_link):

    # Scrape Name
    driver.get(scraped_link)
    scraped_profile_name = wait_for_element_to_load(driver, By.CSS_SELECTOR, "h1.text-heading-xlarge.inline.t-24.v-align-middle.break-words")

    # Scrape Experiences
    experiences_url = os.path.join(scraped_link, "details/experience")
    print(experiences_url)
    driver.get(experiences_url)
    scraped_experiences = get_experiences(driver)
    print("Finished scraping experiences")

    # Scrape Education
    education_url = os.path.join(scraped_link, "details/education")
    print(education_url)
    driver.get(education_url)
    scraped_education = get_education(driver)
    print("Finished scraping education")

    # Scrape degree of connection and mutuals if available
    driver.get(scraped_link)
    scraped_profile_dist, scraped_mutuals = scrape_degree_of_connection_and_mutuals(driver)
    # Scrape description
    scraped_description = scrape_description(driver)

    # Scape profile link
    scraped_link = driver.current_url
    print("Successfully scraped profile link: " + scraped_link)
    
    #TODO: EDIT HERE
    profile = ScrapedProfile(scraped_profile_name,
                   scraped_experiences,
                   scraped_education,
                   scraped_profile_dist,
                   scraped_mutuals,
                   scraped_description,
                   scraped_link)
    return profile

def scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle):
    results = []
    scraped_urls = []

    with open(toScrapePickle, 'rb') as f:
        try:
            to_scrape_urls = pickle.load(f)
        except Exception as e:
            print(e)
            print("Error loading to scrape url pickle file")

    print(f'# of profiles to scrape: {len(to_scrape_urls)}')
    

    for i in range(len(to_scrape_urls) - 1, -1, -1):
        url = to_scrape_urls[i]
        print(f'At index: {len(to_scrape_urls) - i} - url: {url}')

        with open(historyPickle, 'rb') as f:
            try:
                history = pickle.load(f)
            except Exception as e:
                print(e)
                print("Error loading history pickle file")

        try:
            profile = scrape_profile(driver, url)
            if profile != None:
                print("saving profile info")
                results.append(profile)

                # TODO: I can't pickle the profile object. Find workaround like list of dict.
                with open(resultsPickle, 'wb') as f:
                    pickle.dump(results, f)
                
                print("adding to history")
                history.append(url)
                with open(historyPickle, 'wb') as f:
                    pickle.dump(history, f)
                
                print("recording scraped url")
                scraped_urls.append(url)
            else:
                print("profile filtered out")
            
            print("removing from to-scrape")
            to_scrape_urls.remove(url)
            with open(toScrapePickle, 'wb') as f:
                pickle.dump(to_scrape_urls, f)

        except Exception as e:
            print(e)
            print('Failed to scrape profile: ', url)

            # TODO: CHANGED FAILED URLS
            with open('failed_urls.txt', 'a') as f:
                f.write(url + '\n')

        print(((len(to_scrape_urls) - i)/len(to_scrape_urls)) * 100, '% Done - at index:', len(to_scrape_urls) - i)
        print('\n------------------------------------------------------------------------------------------------\n')
    return results

# QUERY 1: MAIN COMPANIES

In [6]:
historyPickle = 'db/main_history.pickle'
toScrapePickle = 'db/main_to_scrape.pickle'
resultsPickle = 'db/main_results.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#coach=false&query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(doLogHistory%3Afalse)%2Cfilters%3AList((type%3APAST_COMPANY%2Cvalues%3AList((id%3Aurn%253Ali%253Aorganization%253A1815218%2Ctext%3AUber%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A309694%2Ctext%3AAirbnb%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2857634%2Ctext%3ACoinbase%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2135371%2Ctext%3AStripe%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A20708%2Ctext%3APalantir%2520Technologies%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3608%2Ctext%3ANVIDIA%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3205573%2Ctext%3ADoorDash%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A675562%2Ctext%3ASquare%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30846%2Ctext%3ASpaceX%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30086%2Ctext%3APalo%2520Alto%2520Networks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3131483%2Ctext%3AFlexport%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3477522%2Ctext%3ADatabricks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A748731%2Ctext%3AKlarna%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3254263%2Ctext%3ARobinhood%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6575553%2Ctext%3AByteDance%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18505670%2Ctext%3ABrex%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2732417%2Ctext%3AInstacart%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17998520%2Ctext%3AScale%2520AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2684737%2Ctext%3APlaid%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3767529%2Ctext%3ANubank%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3991822%2Ctext%3AAirtable%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10043614%2Ctext%3ASnyk%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10607336%2Ctext%3AChainalysis%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A10893210%2Ctext%3Adbt%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11062162%2Ctext%3AGrafana%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11130470%2Ctext%3AOpenAI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11193683%2Ctext%3AHugging%2520Face%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11247457%2Ctext%3ASolugen%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11741116%2Ctext%3ARunway%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A11869260%2Ctext%3ARetool%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1406226%2Ctext%3ARamp%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A14824547%2Ctext%3AFireblocks%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A1594050%2Ctext%3AGoogle%2520DeepMind%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A16181286%2Ctext%3AVercel%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17932068%2Ctext%3ALacework%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A17988315%2Ctext%3ARippling%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18013280%2Ctext%3AFaire%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18293159%2Ctext%3AAnduril%2520Industries%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18309569%2Ctext%3ASemgrep%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18586257%2Ctext%3AAbnormal%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18593641%2Ctext%3AWeights%2520%2526%2520Biases%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18742807%2Ctext%3ATRM%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18769344%2Ctext%3AModern%2520Treasury%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18777798%2Ctext%3ACribl%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18922914%2Ctext%3ADeel%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A19107985%2Ctext%3AMercury%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A24024765%2Ctext%3ACohere%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2418251%2Ctext%3AZapier%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2497653%2Ctext%3ACrowdStrike%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27159855%2Ctext%3AStarburst%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A2850862%2Ctext%3ACanva%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A30898036%2Ctext%3ANotion%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3502352%2Ctext%3AWebflow%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A35462987%2Ctext%3AVanta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3650502%2Ctext%3AFigma%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A37564254%2Ctext%3APersona%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3769390%2Ctext%3ABenchling%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A3954657%2Ctext%3AFivetran%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40671813%2Ctext%3ARobust%2520Intelligence%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A4803356%2Ctext%3ASourcegraph%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6424460%2Ctext%3ASentry%2520%2528sentry.io%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A64890982%2Ctext%3AWiz%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65281968%2Ctext%3ATecton%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A65638805%2Ctext%3AMaterial%2520Security%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A67081245%2Ctext%3ATemporal%2520Technologies%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A68023390%2Ctext%3AIsland%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A68047275%2Ctext%3AUniswap%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A70975817%2Ctext%3AVarda%2520Space%2520Industries%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71668100%2Ctext%3AHadrian%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74126343%2Ctext%3AAnthropic%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74882602%2Ctext%3AGlean%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A7602863%2Ctext%3AZipline%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76262108%2Ctext%3AKumo.AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A80114151%2Ctext%3AClickHouse%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81330326%2Ctext%3AAdept%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81491861%2Ctext%3APredibase%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82318617%2Ctext%3AMidjourney%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A83019124%2Ctext%3AEigen%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89486558%2Ctext%3ACharacter.AI%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89962189%2Ctext%3AThe%2520Arbitrum%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A926041%2Ctext%3AOkta%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A9309408%2Ctext%3ACockroach%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A6453825%2Ctext%3ASamsara%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A92950%2Ctext%3ACisco%2520Meraki%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A101156365%2Ctext%3AManta%2520Network%2520%2528MANTA%2529%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A19104188%2Ctext%3AAvalanche%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81901372%2Ctext%3AOptimism%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A13449964%2Ctext%3APolygon%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A72057622%2Ctext%3ASolana%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18453134%2Ctext%3ASolana%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40769465%2Ctext%3AEthereum%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89970028%2Ctext%3ASui%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A27137803%2Ctext%3AStarkWare%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A74341323%2Ctext%3AOsmosis%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79119792%2Ctext%3AAptos%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A71528119%2Ctext%3ANEAR%2520Protocol%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A40708618%2Ctext%3AInjective%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A76174015%2Ctext%3AMysten%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A94854888%2Ctext%3ASei%2520Network%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A81462746%2Ctext%3ABabylonChain%2520Inc.%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A84802787%2Ctext%3AMonad%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A100996178%2Ctext%3ARitual%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A28405403%2Ctext%3ACelestia%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A82157982%2Ctext%3AAltLayer%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75574122%2Ctext%3AFlashbots%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A69266957%2Ctext%3AAxelar%2520Network%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A80940997%2Ctext%3AAxelar%2520Foundation%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A16181958%2Ctext%3AProtocol%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A89707979%2Ctext%3AEclipse%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79159372%2Ctext%3AEspresso%2520Systems%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A42785598%2Ctext%3AAleo%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A86632358%2Ctext%3ALido%2520Finance%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A75654012%2Ctext%3ALayerZero%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79296193%2Ctext%3ACompound%2520Labs%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A18264732%2Ctext%3AdYdX%2CselectionType%3AINCLUDED%2Cparent%3A(id%3A0))%2C(id%3Aurn%253Ali%253Aorganization%253A79948926%2Ctext%3AdYdX%2520Foundation%2CselectionType%3AINCLUDED)))%2C(type%3ALEAD_INTERACTIONS%2Cvalues%3AList((id%3ALIVP%2Ctext%3AViewed%2520profile%2CselectionType%3AEXCLUDED)%2C(id%3ALIMP%2Ctext%3AMessaged%2CselectionType%3AEXCLUDED)))%2C(type%3AFUNCTION%2Cvalues%3AList((id%3A12%2Ctext%3AHuman%2520Resources%2CselectionType%3AEXCLUDED)%2C(id%3A26%2Ctext%3ACustomer%2520Success%2520and%2520Support%2CselectionType%3AEXCLUDED)%2C(id%3A15%2Ctext%3AMarketing%2CselectionType%3AEXCLUDED)%2C(id%3A3%2Ctext%3AArts%2520and%2520Design%2CselectionType%3AEXCLUDED)%2C(id%3A1%2Ctext%3AAccounting%2CselectionType%3AEXCLUDED)%2C(id%3A2%2Ctext%3AAdministrative%2CselectionType%3AEXCLUDED)))%2C(type%3AYEARS_IN_CURRENT_POSITION%2Cvalues%3AList((id%3A1%2Ctext%3ALess%2520than%25201%2520year%2CselectionType%3AINCLUDED)))%2C(type%3APROFILE_LANGUAGE%2Cvalues%3AList((id%3Aen%2Ctext%3AEnglish%2CselectionType%3AINCLUDED))))%2Ckeywords%3A%2522something%2520new%2522%2520OR%2520%2522stealth%2522)&sessionId=stajGZpuROWIdGr%2BfkPTtA%3D%3D
"""

### Sales nav scraper

In [7]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

Ran out of input
Error loading history pickle file
0
260


In [8]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error clicking three dots button
Error clicking three dots button
Error clicking three dots button
Error clicking three dots button
Error clicking three dots button
Error clicking three dots button
Error clicking three dots button
Timed out waiting for element: class name=artdeco-pagination__button--next
['https://www.linkedin.com/in/dylan-phillips-352662a1', 'https://www.linkedin.com/in/casey-kraft-6ba6028a', 'https://www.linkedin.com/in/keefer-thompson', 'https://www.linkedin.com/in/chris-a-schultz', 'https://www.linkedin.com/in/nick-catalanotto', 'https://www.linkedin.com/in/mira-hall', 'https://www.linkedin.com/in/ariannachurchill', 'https://www.linkedin.com/in/john-thomas-security', 'https://www.linkedin.com/in/laura-lamoreaux-a2a794210', 'https://www.linkedin.com/in/junbeom-chi-230050180', 'https://www.linkedin.com/in/xi-sun-161a5b54', 'https://www.linkedin.com/in/koormann', 'https://www.linkedin.com/in/anderssonrikard', 'https://www.linkedin.com/in/prasadpavuluri', 'https://www.

### Deduplicate

In [9]:
# deduplicate to_scrape_urls
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

Removed 0 duplicates
260


### Profile scraper

In [10]:
# driver.close()
# driver = instantiate_driver()

In [11]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle)
print(len(results))

# of profiles to scrape: 260
At index: 1 - url: https://www.linkedin.com/in/miradu
Ran out of input
Error loading history pickle file
https://www.linkedin.com/in/miradu/details/experience
Found 11 experience items
title: : CEO & cofounder
company: : Blaide · Full-time
dates: : Jan 2024 to Present · 8 mos
location: : San Francisco, California, United States · Hybrid
summary: : Blaide is a stealth AI startup working at the intersection of climate and fintech, in service to home contractors electrifying America's 140M homes.
title: : Vice President of Product Management
company: : Mercury · Full-time
dates: : Feb 2023 to Oct 2023 · 9 mos
location: : San Francisco, California, United States · Hybrid
summary: : As the first Vice President of Product Management at Mercury, I had the opportunity to shape the product management function for the company. Beyond developing customer obsessed innovative strategy, this involved coaching, hiring talented individuals, organizing and leading off-sites

KeyboardInterrupt: 

# QUERY 2: UNICORN COMPANIES

In [ ]:
historyPickle = 'db/unicorn_history.pickle'
toScrapePickle = 'db/unicorn_to_scrape.pickle'
resultsPickle = 'db/unicorn_results.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1813072029&sessionId=yJ9IuYoZT0G8bC7kxxPinA%3D%3D&lipi=urn%3Ali%3Apage%3Ad_sales2_search_people_saved_all%3BNTWWzarcS5mlUquty%2Fuq7A%3D%3D&snfl=uXNCEMFbT%2FSGhBJEgEzwhg%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

driver.close()

### Deduplicate

In [ ]:
# deduplicate to_scrape_urls
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
driver = instantiate_driver()

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle)
print(len(results))

# QUERY 3: ACQUIRED COMPANIES

In [ ]:
historyPickle = 'db/acquired_history.pickle'
toScrapePickle = 'db/acquired_to_scrape.pickle'
resultsPickle = 'db/acquired_results.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1814608365&sessionId=yJ9IuYoZT0G8bC7kxxPinA%3D%3D&lipi=urn%3Ali%3Apage%3Ad_sales2_search_people_saved_all%3BNTWWzarcS5mlUquty%2Fuq7A%3D%3D&snfl=cUBlHyMNTkCja4vGVma26Q%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

driver.close()

### Deduplicate

In [ ]:
# deduplicate to_scrape_urls
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
driver = instantiate_driver()

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle)
print(len(results))

# QUERY 4: VC PORTFOLIO COMPANIES

In [ ]:
historyPickle = 'db/portfolio_history.pickle'
toScrapePickle = 'db/portfolio_to_scrape.pickle'
resultsPickle = 'db/portfolio_results.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people#query=(spellCorrectionEnabled%3Atrue%2CrecentSearchParam%3A(id%3A3873006364%2CdoLogHistory%3Atrue)%2Cfilters%3AList((type%3APAST_COMPANY%2Cvalues%3AList((text%3AKlaviyo%2CselectionType%3AINCLUDED)%2C(text%3ADust%2CselectionType%3AINCLUDED)%2C(text%3AEthos%2520Life%2520Agency%2CselectionType%3AINCLUDED)%2C(text%3AFactory%2CselectionType%3AINCLUDED)%2C(text%3AQuantum%2520Circuits%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AxAI%2CselectionType%3AINCLUDED)%2C(text%3ANeros%2520Technologies%2CselectionType%3AINCLUDED)%2C(text%3AAlkira%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AAPEX%2CselectionType%3AINCLUDED)%2C(text%3AStrongDM%2CselectionType%3AINCLUDED)%2C(text%3AOasis%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AAnrok%2CselectionType%3AINCLUDED)%2C(text%3ACollaborative%2520Robotics%2CselectionType%3AINCLUDED)%2C(text%3AGrow%2520Therapy%2CselectionType%3AINCLUDED)%2C(text%3AZafran%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AFireworks%2520AI%2CselectionType%3AINCLUDED)%2C(text%3AFoundry%2CselectionType%3AINCLUDED)%2C(text%3APhysical%2520Intelligence%2CselectionType%3AINCLUDED)%2C(text%3ARobCo%2CselectionType%3AINCLUDED)%2C(text%3AMeter%2CselectionType%3AINCLUDED)%2C(text%3APennylane%2CselectionType%3AINCLUDED)%2C(text%3AGuardant%2520Health%2CselectionType%3AINCLUDED)%2C(text%3ATacto%2CselectionType%3AINCLUDED)%2C(text%3APrivy%2CselectionType%3AINCLUDED)%2C(text%3AApollo.io%2CselectionType%3AINCLUDED)%2C(text%3ARelease%2CselectionType%3AINCLUDED)%2C(text%3ACaptions%2CselectionType%3AINCLUDED)%2C(text%3AWarp%2CselectionType%3AINCLUDED)%2C(text%3AMach%2520Industries%2CselectionType%3AINCLUDED)%2C(text%3AMedallion%2CselectionType%3AINCLUDED)%2C(text%3ADeno%2CselectionType%3AINCLUDED)%2C(text%3ACybersyn%2CselectionType%3AINCLUDED)%2C(text%3ACaldera%2CselectionType%3AINCLUDED)%2C(text%3ATavus%2CselectionType%3AINCLUDED)%2C(text%3ADagster%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3ABigeye%2CselectionType%3AINCLUDED)%2C(text%3AMutiny%2CselectionType%3AINCLUDED)%2C(text%3AKnowde%2CselectionType%3AINCLUDED)%2C(text%3AStatsig%2CselectionType%3AINCLUDED)%2C(text%3APhysna%2CselectionType%3AINCLUDED)%2C(text%3AStreamlit%2CselectionType%3AINCLUDED)%2C(text%3ANeeva%2CselectionType%3AINCLUDED)%2C(text%3ALoom%2CselectionType%3AINCLUDED)%2C(text%3AAurora%2CselectionType%3AINCLUDED)%2C(text%3ATessian%2CselectionType%3AINCLUDED)%2C(text%3AAmplitude%2CselectionType%3AINCLUDED)%2C(text%3AAthelas%2CselectionType%3AINCLUDED)%2C(text%3AFilecoin%2520Labs%2CselectionType%3AINCLUDED)%2C(text%3AUnity%2CselectionType%3AINCLUDED)%2C(text%3AWhatsApp%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520Meraki%2CselectionType%3AINCLUDED)%2C(text%3ABlock%2CselectionType%3AINCLUDED)%2C(text%3ACisco%2520ThousandEyes%2CselectionType%3AINCLUDED)%2C(text%3ANimble%2520Storage%252C%2520acquired%2520by%2520Hewlett%2520Packard%2520Enterprise%2520company%2520in%25202017%2CselectionType%3AINCLUDED)%2C(text%3AFireEye%252C%2520Inc.%2CselectionType%3AINCLUDED)%2C(text%3AUpwind%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AOnehouse%2CselectionType%3AINCLUDED)%2C(text%3ASeven%2520AI%2CselectionType%3AINCLUDED)%2C(text%3ABedrock%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AWarpStream%2CselectionType%3AINCLUDED)%2C(text%3AKodem%2CselectionType%3AINCLUDED)%2C(text%3ABiogenesis%2CselectionType%3AINCLUDED)%2C(text%3ABraintrust%2CselectionType%3AINCLUDED)%2C(text%3AOpal%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AGreenlite%2CselectionType%3AINCLUDED)%2C(text%3ACensys%2CselectionType%3AINCLUDED)%2C(text%3AApiiro%2CselectionType%3AINCLUDED)%2C(text%3ATome%2CselectionType%3AINCLUDED)%2C(text%3AStackBlitz%2CselectionType%3AINCLUDED)%2C(text%3AObsidian%2520Security%2CselectionType%3AINCLUDED)%2C(text%3AMarqeta%2CselectionType%3AINCLUDED)%2C(text%3AAppDynamics%2CselectionType%3AINCLUDED)%2C(text%3ASprig%253A%2520Eat%2520Well%2CselectionType%3AINCLUDED)%2C(text%3AOpenDNS%2CselectionType%3AINCLUDED)%2C(text%3APure%2520Storage%2CselectionType%3AINCLUDED)%2C(text%3AIBM%2520Instana%2CselectionType%3AINCL
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

driver.close()

### Deduplicate

In [ ]:
# deduplicate to_scrape_urls
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
driver = instantiate_driver()

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle)
print(len(results))

# QUERY 5: INFRA COMPANIES

In [ ]:
historyPickle = 'db/infra_history.pickle'
toScrapePickle = 'db/infra_to_scrape.pickle'
resultsPickle = 'db/infra_results.pickle'

SALES_NAV_SEARCH_URL = """
https://www.linkedin.com/sales/search/people?savedSearchId=1813072029&sessionId=yJ9IuYoZT0G8bC7kxxPinA%3D%3D&lipi=urn%3Ali%3Apage%3Ad_sales2_search_people_saved_all%3BNTWWzarcS5mlUquty%2Fuq7A%3D%3D&snfl=uXNCEMFbT%2FSGhBJEgEzwhg%3D%3D
"""

### Sales nav scraper

In [ ]:
already_scraped_urls, to_scrape_urls = check_pickles(historyPickle, toScrapePickle)

In [ ]:
try:
    updated_to_scrape_urls = scrape_profiles(driver, SALES_NAV_SEARCH_URL, already_scraped_urls, to_scrape_urls, toScrapePickle)
    print(updated_to_scrape_urls)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

driver.close()

### Deduplicate

In [ ]:
# deduplicate to_scrape_urls
deduplicate(toScrapePickle, to_scrape_urls, already_scraped_urls)

### Profile scraper

In [ ]:
driver = instantiate_driver()

In [ ]:
results = scrape_all_profiles(driver, historyPickle, toScrapePickle, resultsPickle)
print(len(results))

# EXPORTING RESULTS

In [ ]:
# TODO: FIX EXPORTS
import pandas as pd

def parseCandidate(x):
    res = {}
    res['url'] = x.profile_link
    res['name'] = x.profile_name
    res['dist'] = x.profile_dist
    res['description'] = x.profile_description

    #Experiences first
    print(x.experiences)
    for i, e in enumerate(x.experiences):
        res[f'exp{i} title'] = e["title: "]
        res[f'exp{i} company'] = e["company: "]
        res[f'exp{i} dates'] = e["dates: "]

    # School second
    for i, e in enumerate(x.profile_school):
        res[f'edu{i} school'] = e["school: "]
        if "degree :" in e:
            res[f'edu{i} degree'] = e["degree: "]

    
    return res

# New Method for Creating df
rows = []
for candidate in results:
    row = parseCandidate(candidate)
    rows.append(row)

df = pd.DataFrame(rows)

#Export to csv
try:
    df.to_csv('candidates.csv', index=False)
    print("Exported to csv")
except:
    print("Failed to export to csv")

#Export to Excel
try:
    df.to_excel('candidates.xlsx', index=False)
    print("Exported to Excel")
except:
    print("Failed to export to Excel")

    
# update db/already_scraped.pickle
with open('db/already_scraped.pickle', 'rb') as f:
    already_scraped = pickle.load(f)
    print(f"Previously scraped: {len(already_scraped)}")
    already_scraped = already_scraped + already_scraped_urls
    already_scraped = list(set(already_scraped))
    print(f"Newly scraped: {len(already_scraped)}")
with open('db/already_scraped.pickle', 'wb') as f:
    pickle.dump(already_scraped, f)

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.file']

def authenticate():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def upload_file_to_drive(file_path, file_name, mime_type):
    creds = authenticate()
    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype=mime_type)

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('File ID: %s' % file.get('id'))

if __name__ == '__main__':
    file_path = 'path_to_your_local_excel_file.xlsx'  # Replace with the path to your local file
    file_name = 'your_excel_file.xlsx'  # Replace with the desired name for the file in Google Drive
    mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

    upload_file_to_drive(file_path, file_name, mime_type)

In [ ]:
categories = COMPANY_CATEGORIES

# Example color mapping for categories
category_colors = {
    "SECURITY": 'red',
    "OTHER": 'blue',
    "PUBLIC": 'green',
    "INFRA": 'yellow',
    "FINTECH": 'orange',
    "CRYPTO": 'purple',
    "FRONTIER": 'cyan',
    "AI": 'magenta'
}

# Create a reverse dictionary for easier lookup: {company: category}
company_category = {}
for category, companies in categories.items():
    for company in companies:
        company_category[company] = category

# Modify the style function
def highlight_by_category(val):
    category = company_category.get(val)
    if category:
        color = category_colors.get(category, 'none')  # default to 'none' if no color is specified
    else:
        color = 'none'
    return f'background-color: {color}'

import re
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
def find_illegal_characters(df):
    for column in df.columns:
        for idx, item in enumerate(df[column]):
            if isinstance(item, str) and ILLEGAL_CHARACTERS_RE.search(item):
                # replace illegal characters with an empty string
                df[column][idx] = ILLEGAL_CHARACTERS_RE.sub('', item)
    return df

styled_df = df
styled_df = find_illegal_characters(styled_df)
styled_df = df.style.applymap(highlight_by_category)

# Save the styled DataFrame to an Excel file
# get today's date in MM-DD-YYYY format
import datetime
now = datetime.datetime.now()
date = now.strftime("%m-%d-%Y")
styled_df.to_excel(f'results/{date}_{start}-{end}.xlsx', engine='openpyxl', index=False)
